   # 《Python编程入门》09 Python spider(爬虫)
   
   ***

<img width=1200 height=480 src="https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1550608634102&di=50b3b0f113ad9adbf28bfc64dde1ff1f&imgtype=0&src=http%3A%2F%2Fimg95.699pic.com%2Fphoto%2F50062%2F4783.jpg_wh860.jpg"/>

## =^^=author:秋叶为何而落 =^=^=^= QQ : 81502787 =^^=

# 一、爬虫是什么？
        简单来说互联网是由一个个站点和网络设备组成的大网，我们通过浏览器访问站点，站点把HTML、JS、CSS代码返回给浏览器，这些代码经过浏览器解析、渲染，将丰富多彩的网页呈现我们眼前.
        如果我们把互联网比作一张大的蜘蛛网，数据便是存放于蜘蛛网的各个节点，而爬虫就是一只小蜘蛛，沿着网络抓取自己的猎物（数据）爬虫指的是：向网站发起请求，获取资源后分析并提取有用数据的程序.
        从技术层面来说就是 通过程序模拟浏览器请求站点的行为，把站点返回的HTML代码/JSON数据/二进制数据（图片、视频） 爬到本地，进而提取自己需要的数据，存放起来使用.

# 二、爬虫的基本流程：
    用户获取网络数据的方式：
    方式1：浏览器提交请求--->下载网页代码--->解析成页面;
    方式2：模拟浏览器发送请求(获取网页代码)->提取有用的数据->存放于数据库或文件中;
    爬虫要做的就是方式2.

    1、发起请求
    使用http库向目标站点发起请求，即发送一个Request
    Request包含：请求头、请求体等 
    Request模块缺陷：不能执行JS 和CSS 代码

    2、获取响应内容
    如果服务器能正常响应，则会得到一个Response
    Response包含：html，json，图片，视频等

    3、解析内容
    解析html数据：正则表达式（RE模块），第三方解析库如Beautifulsoup，pyquery等
    解析json数据：json模块
    解析二进制数据:以wb的方式写入文件

    4、保存数据
    数据库（MySQL，Mongdb、Redis）
    文件

# 三、http协议 请求与响应
    Request：用户将自己的信息通过浏览器（socket client）发送给服务器（socket server）
    Response：服务器接收请求，分析用户发来的请求信息，然后返回数据（返回的数据中可能包含其他链接，如：图片，js，css等）
    ps：浏览器在接收Response后，会解析其内容来显示给用户，而爬虫程序在模拟浏览器发送请求然后接收Response后，是要提取其中的有用数据

# 四、 request
    1、请求方式：
    常见的请求方式：GET / POST

    2、请求的URL
    url全球统一资源定位符，用来定义互联网上一个唯一的资源 例如：一张图片、一个文件、一段视频都可以用url唯一确定

    url编码
    https://www.baidu.com/s?wd=图片
    图片会被编码（看示例代码）

    网页的加载过程是：
    加载一个网页，通常都是先加载document文档，
    在解析document文档的时候，遇到链接，则针对超链接发起下载图片的请求

    3、请求头
    User-agent：请求头中如果没有user-agent客户端配置，服务端可能将你当做一个非法用户host；
    cookies：cookie用来保存登录信息
    请求头需要注意的参数：
    （1）Referrer：访问源至哪里来（一些大型网站，会通过Referrer 做防盗链策略；所有爬虫也要注意模拟）
    （2）User-Agent:访问的浏览器（要加上否则会被当成爬虫程序）
    （3）cookie：请求头注意携带

    4、请求体
    请求体
        如果是get方式，请求体没有内容 （get请求的请求体放在 url后面参数中，直接能看到）
        如果是post方式，请求体是format data

    ps：
    1、登录窗口，文件上传等，信息都会被附加到请求体内
    2、登录，输入错误的用户名密码，然后提交，就可以看到post，正确登录后页面通常会跳转，无法捕捉到post


# 五、 响应Response
    1、响应状态码
      200：代表成功
      301：代表跳转
      404：文件不存在
      403：无权限访问
      502：服务器错误

    2、respone header

    响应头需要注意的参数：
    （1）Set-Cookie:BDSVRTM=0; path=/：可能有多个，是来告诉浏览器，把cookie保存下来；
    （2）Content-Location：服务端响应头中包含Location返回浏览器之后，浏览器就会重新访问另一个页面。

    3、preview就是网页源代码
    JSO数据：如网页html、图片、二进制数据等。 

# 六、总结
    1、总结爬虫流程：
     爬取--->解析--->存储

    2、爬虫所需工具：
     请求库：requests,selenium（可以驱动浏览器解析渲染CSS和JS，但有性能劣势（有用没用的网页都会加载）；）
     解析库：正则，beautifulsoup，pyquery
     存储库：文件，MySQL，Mongodb，Redis
    涉及知识：多线程多进程
    计算密集型任务：使用多进程，因为能Python有GIL，多进程可以利用上CPU多核优势；
    IO密集型任务：使用多线程，做IO切换节省任务执行时间（并发）
    线程池


# 七、 任务：爬取网页图片

In [8]:
from urllib import request
from bs4 import BeautifulSoup
import re
import time

url = "https://zhuanlan.zhihu.com/p/46832759"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
page = request.Request(url, headers=headers)
html = request.urlopen(page).read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')
# print("page")
# 用Beautiful Soup结合正则表达式来提取包含所有图片链接（img标签中，class=**，以.jpg结尾的链接）的语句
links = soup.find_all('img', "origin_image zh-lightbox-thumb", src=re.compile(r'.jpg$'))
print(links)
# 设置保存图片的路径，否则会保存到程序当前路径
path = r'D:\Temp\pic'      # 路径前的r是保持字符串原始值的意思，就是说不对其中的符号进行转义
for link in links:
    print(link.attrs['src'])
    # 保存链接并命名，time.time()返回当前时间戳防止命名冲突
    request.urlretrieve(link.attrs['src'], path+'\%s.jpg' % time.time())   # 使用request.urlretrieve直接将所有远程链接数据下载到本地

[<img class="origin_image zh-lightbox-thumb" data-caption="" data-original="https://pic3.zhimg.com/v2-6e931bcf499f85f1a3bbb2e0e2e6a706_r.jpg" data-rawheight="448" data-rawwidth="696" data-size="normal" src="https://pic3.zhimg.com/v2-6e931bcf499f85f1a3bbb2e0e2e6a706_b.jpg" width="696"/>, <img class="origin_image zh-lightbox-thumb" data-caption="" data-original="https://pic1.zhimg.com/v2-5ce6f83a500005bfb291cab9098fbc24_r.jpg" data-rawheight="608" data-rawwidth="1080" data-size="normal" src="https://pic1.zhimg.com/v2-5ce6f83a500005bfb291cab9098fbc24_b.jpg" width="1080"/>, <img class="origin_image zh-lightbox-thumb" data-caption="" data-original="https://pic2.zhimg.com/v2-2d5f024a958e7c806a82faea8fde3005_r.jpg" data-rawheight="892" data-rawwidth="1000" data-size="normal" src="https://pic2.zhimg.com/v2-2d5f024a958e7c806a82faea8fde3005_b.jpg" width="1000"/>]
https://pic3.zhimg.com/v2-6e931bcf499f85f1a3bbb2e0e2e6a706_b.jpg
https://pic1.zhimg.com/v2-5ce6f83a500005bfb291cab9098fbc24_b.jpg
http

# 八、项目：VIP视频免会员下载

In [ ]:
import requests
from multiprocessing import Pool


def downloadMovie(i):
    # 1.寻找url
    url = "https://sohu.zuida-163sina.com/20181101/DknO8Pek/800kb/hls/iW0Ix4913%03d.ts"%i
    print(url)
    # 模拟浏览器
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"
    }
    # 2.解析url
    r = requests.get(url, headers=headers)
    with open(r'D:\Temp\pic{}'.format(url[-10:]), 'wb') as f:
        f.write(r.content)


pool = Pool(20)
for i in range(1766):
    pool.apply_async(downloadMovie, (i,))

pool.close()
pool.join()


tips:
# 视频拼接：
    在控制台输入：copy /b *.ts xxx.mp4
# 查看网页编码方式：
    在网页控制台输入：document.charset
